In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle('../data/processed/ranking/train_all_row_sample_50.pickle')

In [3]:
sample = df.iloc[:9]

In [7]:
sample.to_pickle('../data/processed/ranking/train_all_row_sample_2_sids.pickle')

In [4]:
with(open('../data/processed/ranking/train_all_row_sample_2_sids.libsvm')) as fo:
    for i in fo:
        print(i)

0 qid:10 1:4 2:47429 3:4604 4:14100

0 qid:10 1:3 2:47429 3:4604 4:700

1 qid:10 1:1 2:48995 3:7396 4:3200

0 qid:10 1:11 2:47796 3:7234 4:3000

0 qid:10 1:8 2:49758 3:6878 4:5500

0 qid:10 1:2 2:49067 3:6345 4:3100

0 qid:21 1:4 2:7667 3:1229 4:2100

1 qid:21 1:2 2:6157 3:1289 4:300

0 qid:21 1:3 2:7667 3:929 4:700



In [11]:
import itertools

import click
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_ranking as tfr
from sklearn.metrics import f1_score

import logging
logger = logging.getLogger(__name__)


tf.enable_eager_execution()
tf.executing_eagerly()

# Store the paths to files containing training and test instances.
# As noted above, we will assume the data is in the LibSVM format
# and that the content of each file is sorted by query ID.

_TRAIN_DATA_PATH = '../data/processed/ranking/train_all_row_sample_2_sids.libsvm'
_TEST_DATA_PATH = '../data/processed/ranking/train_all_row_sample_2_sids.libsvm'

# Define a loss function. To find a complete list of available
# loss functions or to learn how to add your own custom function
# please refer to the tensorflow_ranking.losses module.
_LOSS = "pairwise_logistic_loss"
# _LOSS = "sigmoid_cross_entropy_loss"

_NUM_FEATURES=4

# In the TF-Ranking framework, a training instance is represented
# by a Tensor that contains features from a list of documents
# associated with a single query. For simplicity, we fix the shape
# of these Tensors to a maximum list size and call it "list_size,"
# the maximum number of documents per query in the dataset.
# In this demo, we take the following approach:
#   * If a query has fewer documents, its Tensor will be padded
#     appropriately.
#   * If a query has more documents, we shuffle its list of
#     documents and trim the list down to the prescribed list_size.
_LIST_SIZE = 11

# The total number of features per query-document pair.
# We set this number to the number of features in the MSLR-Web30K
# dataset.

# Parameters to the scoring function.
_BATCH_SIZE = 100
_HIDDEN_LAYER_DIMS = ["20", "10"]


# _OUT_DIR = "../models/tfranking/"

def input_fn(path):
    train_dataset = tf.data.Dataset.from_generator(
        tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
        output_types=(
            {str(k): tf.float32 for k in range(1, _NUM_FEATURES + 1)},
            tf.float32
        ),
        output_shapes=(
            {str(k): tf.TensorShape([_LIST_SIZE, 1])
             for k in range(1, _NUM_FEATURES + 1)},
            tf.TensorShape([_LIST_SIZE])
        )
    )

    train_dataset = train_dataset.batch(_BATCH_SIZE)
    return train_dataset.make_one_shot_iterator().get_next()


In [13]:
sample


,sid,click_time,click_mode,distance_plan,eta,price,transport_mode,plan_time_x,pid,req_time,...,max_temp,min_temp,weather,wind,weather_dy,weather_dyq,weather_q,weather_qdy,weather_xq,weather_xydy
1432355,10,2018-11-10 11:13:24,1,47429,4604,14100.0,4,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432357,10,2018-11-10 11:13:24,1,47429,4604,700.0,3,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432360,10,2018-11-10 11:13:24,1,48995,7396,3200.0,1,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432359,10,2018-11-10 11:13:24,1,47796,7234,3000.0,11,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432358,10,2018-11-10 11:13:24,1,49758,6878,5500.0,8,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1432356,10,2018-11-10 11:13:24,1,49067,6345,3100.0,2,2018-11-10 11:10:36,199899.0,2018-11-10 11:10:36,...,11,0,dy,12,1,0,0,0,0,0
1435574,21,2018-11-10 12:16:36,2,7667,1229,2100.0,4,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0
1435572,21,2018-11-10 12:16:36,2,6157,1289,300.0,2,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0
1435573,21,2018-11-10 12:16:36,2,7667,929,700.0,3,2018-11-10 12:16:01,177401.0,2018-11-10 12:16:01,...,11,0,dy,12,1,0,0,0,0,0


In [12]:
input_fn(_TEST_DATA_PATH)

({'1': <tf.Tensor: id=136, shape=(2, 11, 1), dtype=float32, numpy=
  array([[[11.],
          [ 1.],
          [ 2.],
          [ 8.],
          [ 4.],
          [ 3.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.]],
  
         [[ 4.],
          [ 2.],
          [ 3.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.],
          [ 0.]]], dtype=float32)>,
  '2': <tf.Tensor: id=137, shape=(2, 11, 1), dtype=float32, numpy=
  array([[[47796.],
          [48995.],
          [49067.],
          [49758.],
          [47429.],
          [47429.],
          [    0.],
          [    0.],
          [    0.],
          [    0.],
          [    0.]],
  
         [[ 7667.],
          [ 6157.],
          [ 7667.],
          [    0.],
          [    0.],
          [    0.],
          [    0.],
          [    0.],
          [    0.],
          [    0.],
          [    0.]]], dtype=float32)>,
  '3':